In [1]:
pip install presidio-analyzer presidio-anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 6.5 MB/s eta 0:00:00


In [3]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig

# Initialize engines
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Sample clinical text
text = "Patient John Smith, born on 05/12/1980, was seen at Mercy Hospital on May 20, 2023. Contact: john.smith@email.com, (555) 123-4567."

# Analyze the text
results = analyzer.analyze(text=text, language='en')

# Define the operator properly
operator_config = {"DEFAULT": OperatorConfig("replace", {"new_value": "<PII>"})}

# Anonymize the PII
anonymized_result = anonymizer.anonymize(
    text=text,
    analyzer_results=results,
    operators=operator_config
)

print("Anonymized Text:\n", anonymized_result.text)

Anonymized Text:
 Patient <PII>, born on <PII>, was seen at Mercy Hospital on <PII>. Contact: <PII>, <PII>.


In [8]:

from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json

# Initialize engines
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Define the operator for anonymization
operator_config = {"DEFAULT": OperatorConfig("replace", {"new_value": "<REDACTED>"})}

def anonymize_text(text):
    """Helper function to anonymize a text string"""
    if not isinstance(text, str):
        return text

    results = analyzer.analyze(text=text, language='en')
    anonymized_result = anonymizer.anonymize(
        text=text,
        analyzer_results=results,
        operators=operator_config
    )
    return anonymized_result.text

def anonymize_json_data(json_data):
    """Process each record in the JSON data to anonymize PII"""
    anonymized_data = []

    for record in json_data:
        anonymized_record = {}

        # Anonymize fields that might contain PII
        anonymized_record["patient_id"] = "<REDACTED>"  # Always redact patient_id
        anonymized_record["age"] = record["age"]  # Age is usually safe to keep
        anonymized_record["gender"] = record["gender"]  # Gender is usually safe

        # Process text fields that might contain PII
        anonymized_record["admission_date"] = anonymize_text(record["admission_date"])
        anonymized_record["discharge_date"] = anonymize_text(record["discharge_date"])
        anonymized_record["primary_diagnosis"] = anonymize_text(record["primary_diagnosis"])

        # Process arrays of text
        anonymized_record["procedures"] = [anonymize_text(p) for p in record["procedures"]]
        anonymized_record["medications"] = [anonymize_text(m) for m in record["medications"]]

        # Process free text fields
        anonymized_record["follow_up_instructions"] = anonymize_text(record["follow_up_instructions"])
        anonymized_record["summary"] = anonymize_text(record["summary"])

        anonymized_data.append(anonymized_record)

    return anonymized_data

# Load your JSON data
with open('synthetic_discharge_summaries.json', 'r') as f:
    data = json.load(f)

# Anonymize the data
anonymized_data = anonymize_json_data(data)

# Save the anonymized data
with open('anonymized_discharge_summaries.json', 'w') as f:
    json.dump(anonymized_data, f, indent=2)

print("Anonymization complete. Results saved to anonymized_discharge_summaries.json")

Anonymization complete. Results saved to anonymized_discharge_summaries.json


In [6]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json
from typing import Any, Dict, List, Union
import re
from datetime import datetime

# Initialize engines with enhanced configuration
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Custom operators for different PII types
operators = {
    "DEFAULT": OperatorConfig("replace", {"new_value": "<REDACTED>"}),
    "PERSON": OperatorConfig("replace", {"new_value": "PATIENT"}),
    "DATE_TIME": OperatorConfig("mask", {"chars_to_mask": 10, "masking_char": "X"}),
    "EMAIL_ADDRESS": OperatorConfig("replace", {"new_value": "user@example.com"}),
    "PHONE_NUMBER": OperatorConfig("replace", {"new_value": "XXX-XXX-XXXX"}),
    "US_SSN": OperatorConfig("replace", {"new_value": "XXX-XX-XXXX"}),
    "MEDICAL_LICENSE": OperatorConfig("replace", {"new_value": "ML-XXXXXX"})
}

# Add custom recognizers for medical identifiers
medical_license_recognizer = PatternRecognizer(
    supported_entity="MEDICAL_LICENSE",
    patterns=[r"\b[A-Z]{2}\d{6}\b", r"\b\d{4}-\d{4}-\d{4}\b"]  # Example patterns
)
analyzer.registry.add_recognizer(medical_license_recognizer)

def anonymize_text(text: str, custom_operators: Dict[str, OperatorConfig] = None) -> str:
    """Enhanced text anonymization with custom operators and error handling."""
    if not isinstance(text, str):
        return text

    try:
        results = analyzer.analyze(
            text=text,
            language='en',
            entities=["PERSON", "DATE_TIME", "EMAIL_ADDRESS", "PHONE_NUMBER",
                      "US_SSN", "MEDICAL_LICENSE", "LOCATION", "ORGANIZATION"],
            score_threshold=0.6
        )

        anonymized_result = anonymizer.anonymize(
            text=text,
            analyzer_results=results,
            operators=custom_operators if custom_operators else operators
        )
        return anonymized_result.text
    except Exception as e:
        print(f"Error anonymizing text: {str(e)}")
        return text  # Return original if anonymization fails

def process_dates(value: Any) -> Any:
    """Process date fields to maintain utility while reducing identifiability."""
    if not isinstance(value, str):
        return value

    try:
        # Convert to datetime and keep only year if full date is not essential
        dt = datetime.strptime(value, "%Y-%m-%d")
        return str(dt.year)  # Return just the year
    except (ValueError, TypeError):
        return anonymize_text(value)

def anonymize_nested_data(data: Union[Dict, List, str]) -> Union[Dict, List, str]:
    """Recursively process nested JSON structures."""
    if isinstance(data, dict):
        return {k: anonymize_nested_data(v) for k, v in data.items()}
    elif isinstance(data, list):
        return [anonymize_nested_data(item) for item in data]
    elif isinstance(data, str):
        return anonymize_text(data)
    else:
        return data

def anonymize_json_data(json_data: List[Dict]) -> List[Dict]:
    """Process clinical records with field-specific anonymization rules."""
    anonymized_data = []

    for record in json_data:
        if not isinstance(record, dict):
            anonymized_data.append(record)
            continue

        anonymized_record = {}

        # Direct identifiers - always redact
        anonymized_record["patient_id"] = "<REDACTED>"
        anonymized_record["medical_record_number"] = "<MRN_REDACTED>"

        # Demographic information - can be generalized
        anonymized_record["age"] = f"{int(record['age']//10)*10}-{int(record['age']//10)*10+9}" if isinstance(record.get('age'), (int, float)) else record.get('age')
        anonymized_record["gender"] = record.get("gender", "").upper()[:1]  # Just 'M' or 'F'

        # Date processing
        anonymized_record["admission_date"] = process_dates(record.get("admission_date"))
        anonymized_record["discharge_date"] = process_dates(record.get("discharge_date"))

        # Clinical information - anonymize only PII, keep medical terms
        anonymized_record["primary_diagnosis"] = anonymize_text(record.get("primary_diagnosis", ""))
        anonymized_record["secondary_diagnoses"] = [anonymize_text(d) for d in record.get("secondary_diagnoses", [])]

        # Procedures and medications - keep clinical terms but remove identifiers
        anonymized_record["procedures"] = [anonymize_text(p) for p in record.get("procedures", [])]
        anonymized_record["medications"] = [re.sub(r'\b\d+mg\b', 'XXmg', anonymize_text(m)) for m in record.get("medications", [])]

        # Free text fields - need careful processing
        anonymized_record["summary"] = anonymize_text(record.get("summary", ""))
        anonymized_record["follow_up_instructions"] = anonymize_text(record.get("follow_up_instructions", ""))

        # Handle nested structures
        if "provider_info" in record:
            anonymized_record["provider_info"] = {
                "name": "<PROVIDER>",
                "department": record["provider_info"].get("department", "")
            }

        anonymized_data.append(anonymized_record)

    return anonymized_data

def validate_json_schema(data: List[Dict]) -> bool:
    """Basic validation of expected clinical JSON schema."""
    required_fields = {"patient_id", "age", "summary"}
    return all(required_fields.issubset(record.keys()) for record in data if isinstance(record, dict))

# Main execution
if __name__ == "__main__":
    try:
        # Load and validate JSON data
        with open('synthetic_discharge_summaries.json', 'r') as f:
            data = json.load(f)

        if not validate_json_schema(data):
            raise ValueError("Input JSON does not match expected schema")

        # Anonymize the data
        anonymized_data = anonymize_json_data(data)

        # Save the anonymized data
        with open('anonymized_discharge_summaries.json', 'w') as f:
            json.dump(anonymized_data, f, indent=2)

        print("Anonymization complete. Results saved to anonymized_discharge_summaries.json")

    except json.JSONDecodeError:
        print("Error: Invalid JSON file format")
    except FileNotFoundError:
        print("Error: Input file not found")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

...

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'


Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error 

Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Error anonymizing text: 'str' object has no attribute 'compiled_regex'
Anonymization complete. Results saved to anonymized_discharge_summaries.json


In [7]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
import json
import uuid

# Initialize engines
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

# Create a mapping store for reversible anonymization
token_mapping = {}

def generate_token():
    return str(uuid.uuid4())

def anonymize_with_mapping(text):
    """Anonymize text but store original values with tokens"""
    if not isinstance(text, str):
        return text

    results = analyzer.analyze(text=text, language='en')
    anonymized_result = anonymizer.anonymize(
        text=text,
        analyzer_results=results,
        operators={"DEFAULT": OperatorConfig("replace", {"new_value": generate_token()})}
    )

    # Store mapping between tokens and original values
    for result in results:
        original_value = text[result.start:result.end]
        token = anonymized_result.text[result.start:result.end]
        token_mapping[token] = original_value

    return anonymized_result.text

def deanonymize_with_mapping(text):
    """Restore original values using token mapping"""
    if not isinstance(text, str):
        return text

    for token, original_value in token_mapping.items():
        text = text.replace(token, original_value)
    return text

# Example usage
original_data = [{
    "patient_id": "123-45-6789",
    "name": "John Doe",
    "diagnosis": "Hypertension"
}]

# Anonymize
anonymized_data = []
for record in original_data:
    anonymized_record = {
        "patient_id": anonymize_with_mapping(record["patient_id"]),
        "name": anonymize_with_mapping(record["name"]),
        "diagnosis": record["diagnosis"]  # Non-PII remains
    }
    anonymized_data.append(anonymized_record)

# Deanonymize
deanonymized_data = []
for record in anonymized_data:
    deanonymized_record = {
        "patient_id": deanonymize_with_mapping(record["patient_id"]),
        "name": deanonymize_with_mapping(record["name"]),
        "diagnosis": record["diagnosis"]
    }
    deanonymized_data.append(deanonymized_record)

print("Original:", original_data)
print("Anonymized:", anonymized_data)
print("Deanonymized:", deanonymized_data)

Original: [{'patient_id': '123-45-6789', 'name': 'John Doe', 'diagnosis': 'Hypertension'}]
Anonymized: [{'patient_id': '41115eea-2f0f-41f8-bef7-97c20a30f5cf', 'name': 'f9772dab-0df1-4e40-949b-3b8421889b0e', 'diagnosis': 'Hypertension'}]
Deanonymized: [{'patient_id': '123-45-67890f-41f8-bef7-97c20a30f5cf', 'name': 'John Doe-0df1-4e40-949b-3b8421889b0e', 'diagnosis': 'Hypertension'}]
